<a href="https://colab.research.google.com/github/ishahmshah1025/Mindspark-24/blob/main/twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

pip install git+https://github.com/tweepy/tweepy.git

  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-ypfxbu62
  Running command git clone --filter=blob:none --quiet https://github.com/tweepy/tweepy.git /tmp/pip-req-build-ypfxbu62
  Resolved https://github.com/tweepy/tweepy.git to commit 91a41c6e1c955d278c370d51d5cf43b05f7cd979
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-fzec603t
  Running command git clone --filter=blob:none --quiet https://github.com/tweepy/tweepy.git /tmp/pip-req-build-fzec603t
  Resolved https://github.com/tweepy/tweepy.git to commit 91a41c6e1c955d278c370d51d5cf43b05f7cd979
  Preparing metadata (setup.py) ... done


In [6]:
pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.5 MB/s eta 0:00:00


In [7]:
import tweepy
import pandas as pd



consumer_key = 'EIKSrLVqeRhvb0F6QA93YuudO'
consumer_secret = 'IiMBO9gXqM5ixC6xWikTuVO7VgiinlD36onjONSrcXmdmePTTO'
access_token = '1847140932186980352-7wFaPuKaMgiGpWYAa6mt8vFbClpFxL'
access_token_secret = 'hscAkIbBBMqjvI6ppUbhOOkVkbt2nnOI5cpJKMhI6FNiX'


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


def fetch_tweets(stock_name, count=100):
    tweets = api.search_tweets(q=stock_name, count=count, lang='en')
    tweet_list = [[tweet.text, tweet.created_at] for tweet in tweets]
    return pd.DataFrame(tweet_list, columns=['Tweet', 'Date'])

In [8]:
tweets = api.search_tweets(q='AAPL', count=10, lang='en')
print(tweets)


Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product

In [5]:
import re

def clean_tweet(tweet):
    tweet = re.sub(r'@[A-Za-z0-9_]+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/\S+', '', tweet)
    return tweet


def analyze_sentiment(df):
    analyzer = SentimentIntensityAnalyzer()

    def get_sentiment(tweet):
        vs = analyzer.polarity_scores(tweet)
        if vs['compound'] >= 0.05:
            return 'Positive'
        elif vs['compound'] <= -0.05:
            return 'Negative'
        else:
            return 'Neutral'

    df['Sentiment'] = df['Tweet'].apply(clean_tweet).apply(get_sentiment)
    return df


def aggregate_sentiments(df):
    sentiment_counts = df['Sentiment'].value_counts()
    return {
        'Positive': sentiment_counts.get('Positive', 0),
        'Negative': sentiment_counts.get('Negative', 0),
        'Neutral': sentiment_counts.get('Neutral', 0)
    }

from flask import Flask, jsonify, request, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/analyze', methods=['GET'])
def analyze_stock_sentiment():
    stock_name = request.args.get('stock')
    df = fetch_tweets(stock_name)
    df = analyze_sentiment(df)
    sentiment_counts = aggregate_sentiments(df)

    return jsonify(sentiment_counts)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
